# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:**   
Based on the papers above, I choose hidden_size = 512, embed_size = 256.  
Batch_size: I think the range between 128 and 512 is reasonable, i choose 256 based on my favor and my sense.  
vocal_threshold:  4 is a trade-off value  between low frequence and  high frequence.  
vocab_from_file: accelarate to load   
num_epochs:  time factor. I will choose bigger num_epochs, adapt learning rate for achieve better result if I have plenty of spare time.  
### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:**  
It is a normal transform in almost image tasks.   
Image normalization is the critical for input in CNN network, after normaliztion operation step the image data conforms to the Gaussian distribution.  
the image transform can add datat volume and boost the network generalization.   

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:**  
```
params = [{'params': encoder.embed.parameters()}, 
          {'params': decoder.parameters(), 'lr': 1e-2}]
```
Pre-train resnet50 network used to extract features from input image. The resnet50 network be trained in Imagenet Dataset, and it is benificial to use the network to refine abstract feature(concise features), so I think the network params do not need to fine-tuning.    
The weights in the embedding layer of the encoder and the weights in the deconder just initialization randomly, so the all weights need to be trained for adapt to the special task.

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:**  
I choose adam optimizer algorithm, because its propose based on Momentum and RMSProp. In facial keypoints detection project, it easily converge better point than  other algorithms.  
During the traning,  i choose adapt learning rate for getting better result.

In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
#sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math

'''
## TODO #1: Select appropriate values for the Python variables below.
batch_size = 128          # batch size
vocab_threshold = 4        # minimum word count threshold
vocab_from_file = False    # if True, load existing vocab file
embed_size = 256           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 2             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity
'''
## TODO #1: Select appropriate values for the Python variables below.
batch_size = 30          # batch size
vocab_threshold = 3        # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 512           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 1             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
num_layers = 3
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity
# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size, num_layers)

encoder.train()
decoder.train()

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)


# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
#params = [{'params': encoder.embed.parameters()}, 
#          {'params': decoder.parameters(), 'lr': 1e-3}]
params = list(decoder.parameters()) + list(encoder.embed.parameters())
# TODO #4: Define the optimizer.
optimizer = torch.optim.Adam(params, lr = 0.00092)
#optimizer = torch.optim.Adam(params)
# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

print("total_step = {}".format(total_step))

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=1.61s)
creating index...
index created!
Obtaining caption lengths...


100%|████████████████████████████████████████████████████████████████████████| 414113/414113 [00:56<00:00, 7390.46it/s]


total_step = 13804


<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [ ]:
# Print the total number of keys in the word2idx dictionary.
print('Total number of tokens in vocabulary:', len(data_loader.dataset.vocab))

In [ ]:
import os
encoder_file = "encoder-1.pkl"
decoder_file = "decoder-1.pkl"
encoder.load_state_dict(torch.load(os.path.join('.\\models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('.\\models', decoder_file)))

In [ ]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')
'''
old_time = time.time()
response = requests.request("GET", 
                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                            headers={"Metadata-Flavor":"Google"})
'''
lr = 0.00021
start = 1
for epoch in range(start, num_epochs+start):
    optimizer = torch.optim.Adam(params, lr = lr)
    for i_step in range(1, total_step+1):
        
        '''
        if time.time() - old_time > 60:
            old_time = time.time()
            requests.request("POST", 
                             "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                             headers={'Authorization': "STAR " + response.text})
        '''
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))
        #print("obtain data")
        #print("images.shape = {}, captions.shape = {}".format(images.shape, captions.shape))
        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        '''
        print("captions is {}".format(captions[:2,:]))
        batch, n_seq = captions.shape
        caption = torch.zeros(batch, n_seq, vocab_size).scatter_( 2, \
                              captions.cpu().view(batch, n_seq, -1), 1.0)
        print(captions.cpu().view(batch, n_seq, -1)[0,:,:])
        print("one hot encode captions : {}".format(caption[:2,:5,:30]))
        '''
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        #print("outputs.shape = {}, captions.shape = {}".format(outputs, captions.shape))
        
        # Calculate the batch loss.
        # output = outputs.permute() 
        # print("outputs.shape = {}, captions.shape = {}".format(outputs.contiguous().shape, captions.view(-1).shape))
        
        loss = criterion(outputs.contiguous().view(-1, vocab_size), captions.view(-1))
        
        # print(loss.item())
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % \
        (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))

        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.write("predict: {0} \n".format(torch.argmax(outputs, 2)[:1,:]))
        f.write("true lable:{0} \n".format(captions[:1,:]))
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:        
            print('\r' + stats)
            print(" predict:   ", torch.argmax(outputs, 2)[:1,:])
            print("true lable: ", captions[:1,:])
        if i_step in [int(j*total_step) for j in [0.33*i for i in range(1,3)]]:
            lr = lr * 0.21
            optimizer = torch.optim.Adam(params, lr = lr)
            print(optimizer.state_dict)
        if i_step % (print_every*10) == 0:
            torch.save(encoder.state_dict(), os.path.join('.\\models', 'encoder-%d_%d.pkl' % (epoch, i_step)))
            torch.save(decoder.state_dict(), os.path.join('.\\models', 'decoder-%d_%d.pkl' % (epoch, i_step)))
        if i_step == 10001:
            break
        #break
    #break
    
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(encoder.state_dict(), os.path.join('.\\models', 'encoder-%d.pkl' % epoch))
        torch.save(decoder.state_dict(), os.path.join('.\\models', 'decoder-%d.pkl' % epoch))
    
        

# Close the training log file.
f.close()

Epoch [1/1], Step [100/13804], Loss: 2.6927, Perplexity: 14.7713
 predict:    tensor([[    0,     3,   130,     3,  2515,    13,     3,  2515,    13,
          4212,    18,    77,    18]], device='cuda:0')
true lable:  tensor([[    0,   145,   130,     3,   407,    13,     3,  3278,    13,
          4170,    53,   986,     1]], device='cuda:0')
Epoch [1/1], Step [200/13804], Loss: 2.0931, Perplexity: 8.11006
 predict:    tensor([[    0,     3,   130,     3,   170,    39,   130,   361,  1512,
            18,    18,    18]], device='cuda:0')
true lable:  tensor([[     0,    251,    130,      3,    170,     64,    130,    356,
          10756,    523,    354,      1]], device='cuda:0')
Epoch [1/1], Step [300/13804], Loss: 2.7941, Perplexity: 16.3474
 predict:    tensor([[   0,    3,  829,  829,   21,    3,   39,    3,  112,   18,
            1]], device='cuda:0')
true lable:  tensor([[    0,     3,    20,   829,     6,  3232,    39,     3,   112,
            18,     1]], device='cuda:0')


             3,   210,   211,    18,     1]], device='cuda:0')
Epoch [1/1], Step [2500/13804], Loss: 2.1509, Perplexity: 8.59236
 predict:    tensor([[   0,    3,  170,  295,   13,    3,   51,  225,  111,    3,
          315,  158,   18]], device='cuda:0')
true lable:  tensor([[   0,    3,  651,  295,   13,   50,   51,  286,  111,    3,
          315,  158,    1]], device='cuda:0')
Epoch [1/1], Step [2600/13804], Loss: 2.6024, Perplexity: 13.4965
 predict:    tensor([[   0,    3,  829,  581,   21,   21,    3,    6,   39,    3,
          112,   18,    1]], device='cuda:0')
true lable:  tensor([[    0,     3,  2411,  1606,  1186,    21,  3496,   225,    39,
             3,   112,    18,     1]], device='cuda:0')
Epoch [1/1], Step [2700/13804], Loss: 2.4045, Perplexity: 11.0729
 predict:    tensor([[    0,     3,   829,    13,  2107,    21,    39,     3,   829,
           829,    18,     1]], device='cuda:0')
true lable:  tensor([[    0,     3,   934,    13,  3555,   327,    39,     3,   

            3,  229,   33,   18,    1]], device='cuda:0')
Epoch [1/1], Step [4791/13804], Loss: 2.1242, Perplexity: 8.36621

In [ ]:
encoder

In [ ]:
decoder

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.